# Setup
(No need to read)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    # Needed for PySvelte to work, v3 came out and broke things...
    %pip install typeguard==2.13.3
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-2kp2_mtn
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-2kp2_mtn
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 10d2f8a026d73eada861c7d51064f7e24d8f482c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━


## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache...

+ apt-get update
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [43.3 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [850 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,235 kB]
Hit:12 https

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
from dataclasses import dataclass
import datasets
from IPython.display import HTML

In [4]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [5]:
import plotly.graph_objects as go

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth"}
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0,labels={"x":xaxis, "y":yaxis}, **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label

    fig.show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(y=utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    # Helper function to plot multiple lines
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

def bar(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.bar(
        y=utils.to_numpy(tensor),
        labels={"x": xaxis, "y": yaxis},
        template="simple_white",
        **kwargs).show(renderer)

In [6]:
import transformer_lens.patching as patching
from transformer_lens import evals
import math

In [7]:
def visualize_attn_patterns(heads, local_tokens, local_cache, title: str = ""):
    labels = []
    patterns = []
    batch_index = 0

    for head in heads:
        if isinstance(head, tuple):
            layer, head_index = head
        else:
            layer, head_index = head // model.cfg.n_heads, head % model.cfg.n_heads
        patterns.append(local_cache["pattern", layer][batch_index, head_index])
        labels.append(f"L{layer}H{head_index}")
    patterns = torch.stack(patterns, dim=-1)
    attn_viz = pysvelte.AttentionMulti(tokens=model.to_str_tokens(local_tokens[batch_index]), attention=patterns, head_labels=labels)
    display(HTML(f"<h3>{title}</h3>"))
    attn_viz.show()

# Load Model

In [8]:
torch.set_grad_enabled(True)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [10]:
LIST_LEN = 10
MAX_NUM = 50
cfg = HookedTransformerConfig(
    n_layers=1,
    d_model=128,
    d_head=128,
    n_ctx=LIST_LEN*2 + 2, # BOS 1 4 2 MID 1 2 4
    d_vocab=MAX_NUM+2, # 0, 1, ..., MAX_NUM-1, BOS, MID
    d_vocab_out=MAX_NUM,
    attn_only=True,
    normalization_type=None,
    device=device,
    seed=0
)

model = HookedTransformer(cfg)
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)


In [11]:
# disable biases
for name, param in model.named_parameters():
    if 'b_' in name:
        param.requires_grad = False
    print(name, param.shape, param.requires_grad)

embed.W_E torch.Size([52, 128]) True
pos_embed.W_pos torch.Size([22, 128]) True
blocks.0.attn.W_Q torch.Size([1, 128, 128]) True
blocks.0.attn.W_K torch.Size([1, 128, 128]) True
blocks.0.attn.W_V torch.Size([1, 128, 128]) True
blocks.0.attn.W_O torch.Size([1, 128, 128]) True
blocks.0.attn.b_Q torch.Size([1, 128]) False
blocks.0.attn.b_K torch.Size([1, 128]) False
blocks.0.attn.b_V torch.Size([1, 128]) False
blocks.0.attn.b_O torch.Size([128]) False
unembed.W_U torch.Size([128, 50]) True
unembed.b_U torch.Size([50]) False


# Training Code

## Task Dataset

In [12]:
def make_data_generator(cfg, batch_size, seed=0):
    torch.manual_seed(seed)
    MID_TOKEN = cfg.d_vocab-1
    BOS_TOKEN = cfg.d_vocab-2
    while True:
        x = torch.randint(0, cfg.d_vocab_out, (batch_size, LIST_LEN))
        sorted_x = x.sort(dim=-1).values
        mid_vec = torch.ones(batch_size).unsqueeze(-1) * MID_TOKEN
        bos_vec = torch.ones(batch_size).unsqueeze(-1) * BOS_TOKEN

        tokens = torch.cat([bos_vec, x, mid_vec, sorted_x], dim=-1).to(torch.long)
        yield tokens

print(next(make_data_generator(cfg, 2)))

tensor([[50, 44, 39, 33, 10, 13, 29, 27,  3, 47, 33, 51,  3, 10, 13, 27, 29, 33,
         33, 39, 44, 47],
        [50,  1, 16,  6, 49, 28, 26,  6, 18, 44, 33, 51,  1,  6,  6, 16, 18, 26,
         28, 33, 44, 49]])


## Loss Fn

In [13]:
def loss_fn(logits, tokens):
    logits = logits[:, -(LIST_LEN+1):-1, :]
    log_probs = logits.log_softmax(dim=-1)
    labels = tokens[:, -LIST_LEN:]
    correct_log_probs = log_probs.gather(dim=-1, index=labels[..., None])[..., 0]
    return -correct_log_probs.mean()

with torch.no_grad():
    tokens = next(make_data_generator(cfg, 2))
    tokens = tokens.to(device)
    logits = model(tokens)
    loss = loss_fn(logits, tokens)
    print(loss)

tensor(3.9422, device='cuda:0')


In [14]:
print("uniform loss:", np.log(cfg.d_vocab_out))

uniform loss: 3.912023005428146


## Setup optimizer / dataloader

In [15]:
lr = 1e-3
wd = 0.01
betas = (0.9, 0.98)

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=betas)

batch_size = 256
train_data_loader = make_data_generator(cfg, batch_size)

## Training Loop

In [16]:
num_epochs = 10000

train_losses = []
for epoch in tqdm.tqdm(range(num_epochs)):
    tokens = next(train_data_loader)
    tokens = tokens.to(device)

    logits = model(tokens)
    loss = loss_fn(logits, tokens)
    loss.backward()
    train_losses.append(loss.item())

    optimizer.step()
    optimizer.zero_grad()

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, train_loss: {loss.item()}")

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 0, train_loss: 3.9172489643096924
Epoch: 100, train_loss: 2.3987338542938232
Epoch: 200, train_loss: 1.2190136909484863
Epoch: 300, train_loss: 0.4392028748989105
Epoch: 400, train_loss: 0.3081232011318207
Epoch: 500, train_loss: 0.22951793670654297
Epoch: 600, train_loss: 0.17295096814632416
Epoch: 700, train_loss: 0.16446828842163086
Epoch: 800, train_loss: 0.1401982307434082
Epoch: 900, train_loss: 0.15263152122497559
Epoch: 1000, train_loss: 0.1349060982465744
Epoch: 1100, train_loss: 0.13596658408641815
Epoch: 1200, train_loss: 0.12159226089715958
Epoch: 1300, train_loss: 0.10960417985916138
Epoch: 1400, train_loss: 0.1079125627875328
Epoch: 1500, train_loss: 0.09948201477527618
Epoch: 1600, train_loss: 0.10766289383172989
Epoch: 1700, train_loss: 0.10108118504285812
Epoch: 1800, train_loss: 0.12263192981481552
Epoch: 1900, train_loss: 0.10367965698242188
Epoch: 2000, train_loss: 0.11788195371627808
Epoch: 2100, train_loss: 0.10300252586603165
Epoch: 2200, train_loss: 0.101

## Sanity check

In [17]:
test_data = next(train_data_loader).to(device)
with torch.inference_mode():
    logits = model(test_data)
    logits = logits[:, -(LIST_LEN+1):-1, :]
    preds = logits.argmax(dim=-1)
    labels = test_data[:, -LIST_LEN:]
    acc = (preds == labels).float().mean()
    print("accuracy on test sample:", acc.item())

accuracy on test sample: 0.9957031607627869


## Save model

In [18]:
%mkdir ../models

In [19]:
filename = "../models/sort_fixed_len_list_model.pt"
torch.save(model.state_dict(), filename)

In [20]:
# sanity check we can load model

LIST_LEN = 10
MAX_NUM = 50
cfg = HookedTransformerConfig(
    n_layers=1,
    d_model=128,
    d_head=128,
    n_ctx=LIST_LEN*2 + 2, # BOS 1 4 2 MID 1 2 4
    d_vocab=MAX_NUM+2, # 0, 1, ..., MAX_NUM-1, BOS, MID
    d_vocab_out=MAX_NUM,
    attn_only=True,
    normalization_type=None,
    device=device,
    seed=0
)

loaded_model = HookedTransformer(cfg)

state_dict = torch.load(filename)
loaded_model.load_state_dict(state_dict, strict=True)

<All keys matched successfully>